## Домашнее задание к занятию 3: "Логистическая регрессия. Log Loss"

2. Подберите аргументы функции eval_model для логистической регрессии таким образом, чтобы log loss был минимальным.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression

In [2]:
X = np.array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
              [1, 1, 2, 1, 3, 0, 5, 10, 1, 2],
              [500, 700, 750, 600, 1450,
               800, 1500, 2000, 450, 1000],
              [1, 1, 2, 1, 2, 1, 3, 3, 1, 2]], dtype = np.float64)

y = np.array([0, 0, 1, 0, 1, 0, 1, 0, 1, 1], dtype = np.float64)

In [3]:
def calc_logloss(y, y_pred):
    err = - np.mean(y * np.log(y_pred) + (1.0 - y) * np.log(1.0 - y_pred))
    return err

In [4]:
def sigmoid(z):
    res = 1 / (1 + np.exp(-z))
    return res

In [5]:
def eval_model(X, y, iterations, alpha=1e-4, metric: callable = None):
    np.random.seed(42)
    W = np.random.randn(X.shape[0])
    n = X.shape[1]
    for i in range(1, iterations+1):
        z = np.dot(W, X)
        y_pred = sigmoid(z)
        err = calc_logloss(y, y_pred)
        W -= alpha * (1/n * np.dot((y_pred - y), X.T))
#     if i % (iterations / 10) == 0:
#         print(i, W, err)
    return W, err

In [6]:
iter_array = [1000, 5000, 10000, 100000, 200000, 300000, 400000, 500000, 1000000]
alpha_array = [1e-2, 1e-3, 1e-4, 1e-5, 1e-6, 1e-7, 1e-8]

In [7]:
for i in range(len(iter_array)):
    for j in range(len(alpha_array)):
        W, err = eval_model(X, y, iterations=iter_array[i], alpha=alpha_array[j])
        print('iterations=', iter_array[i], 'alpha=', alpha_array[j], 'error', err)

<ipython-input-3-7d5907c1794a>:2: RuntimeWarning: divide by zero encountered in log
  err = - np.mean(y * np.log(y_pred) + (1.0 - y) * np.log(1.0 - y_pred))
<ipython-input-3-7d5907c1794a>:2: RuntimeWarning: invalid value encountered in multiply
  err = - np.mean(y * np.log(y_pred) + (1.0 - y) * np.log(1.0 - y_pred))
<ipython-input-4-304f158d4adf>:2: RuntimeWarning: overflow encountered in exp
  res = 1 / (1 + np.exp(-z))


iterations= 1000 alpha= 0.01 error nan
iterations= 1000 alpha= 0.001 error nan
iterations= 1000 alpha= 0.0001 error 5.042792352603376
iterations= 1000 alpha= 1e-05 error 0.8478082963339114
iterations= 1000 alpha= 1e-06 error nan
iterations= 1000 alpha= 1e-07 error nan
iterations= 1000 alpha= 1e-08 error nan
iterations= 5000 alpha= 0.01 error nan
iterations= 5000 alpha= 0.001 error 170.8256709756176
iterations= 5000 alpha= 0.0001 error 2.3207568321112637
iterations= 5000 alpha= 1e-05 error 0.8471318040263481
iterations= 5000 alpha= 1e-06 error 0.6047007196667514
iterations= 5000 alpha= 1e-07 error nan
iterations= 5000 alpha= 1e-08 error nan
iterations= 10000 alpha= 0.01 error nan
iterations= 10000 alpha= 0.001 error nan
iterations= 10000 alpha= 0.0001 error 1.3037108262994646
iterations= 10000 alpha= 1e-05 error 0.846289770586375
iterations= 10000 alpha= 1e-06 error 0.6046369485072736
iterations= 10000 alpha= 1e-07 error nan
iterations= 10000 alpha= 1e-08 error nan
iterations= 100000 al

Можно заметить, что наилучшие результаты получаются при скорости $e^{-6}$. При увеличении количества итераций ошибка уменьшается, но при количестве итераций больше $e^6$ сильно увеличивается время выполнения программы, поэтому оптимальными будут следующие значения:
* $\alpha = e^{-6}$
* Количество итераций = 1000000

При необходимости увеличения точности можно увеличить количество итераций. 

In [8]:
W, error = eval_model(X, y, iterations=1000000, alpha=1e-6)
W, error

<ipython-input-3-7d5907c1794a>:2: RuntimeWarning: divide by zero encountered in log
  err = - np.mean(y * np.log(y_pred) + (1.0 - y) * np.log(1.0 - y_pred))
<ipython-input-3-7d5907c1794a>:2: RuntimeWarning: invalid value encountered in multiply
  err = - np.mean(y * np.log(y_pred) + (1.0 - y) * np.log(1.0 - y_pred))


(array([ 0.418086  , -0.09610316, -0.00257571,  1.56844584]),
 0.5944741216714555)

3. Создайте функцию calc_pred_proba, возвращающую предсказанную вероятность класса 1 (на вход подаются W, который уже посчитан функцией eval_model и X, на выходе - массив y_pred_proba).

In [9]:
def calc_pred_proba(W, X):
    y_pred_proba = sigmoid(np.dot(W, X))
    return y_pred_proba

In [10]:
y_pred_proba = calc_pred_proba(W, X)
y_pred_proba

array([0.64624487, 0.52184396, 0.80705531, 0.58540518, 0.38506328,
       0.48149936, 0.68553835, 0.27110399, 0.67510344, 0.68719994])

4. Создайте функцию calc_pred, возвращающую предсказанный класс (на вход подаются W, который уже посчитан функцией eval_model и X, на выходе - массив y_pred).

In [11]:
def calc_pred(W, X):
    y_pred = []
    y_pred_sigm = calc_pred_proba(W, X)
    for elem in y_pred_sigm:
        if elem >= 0.5:
            y_pred.append(1)
        else:
            y_pred.append(0)
    return y_pred

In [12]:
y_pred = calc_pred(W, X)
y_pred

[1, 1, 1, 1, 0, 0, 1, 0, 1, 1]

5. Посчитайте Accuracy, матрицу ошибок, точность и полноту, а также F1 score.

In [13]:
positiv = 0
for i in range(len(y_pred)):
    if y_pred[i] == y[i]:
        positiv += 1
accuracy = positiv / len(y)
accuracy

0.6

In [14]:
y

array([0., 0., 1., 0., 1., 0., 1., 0., 1., 1.])

In [15]:
y_pred

[1, 1, 1, 1, 0, 0, 1, 0, 1, 1]

In [16]:
TN = 0
TP = 0
FN = 0
FP = 0
for i in range(len(y)):
    print('y_real:', y[i], '   y_pred:', y_pred[i])
    if y_pred[i] == y[i]:
        if y[i] == 0:
            TN += 1
            
        else:
            TP += 1
    else:
        if y_pred[i] == 1.0:
            FP += 1
        else:
            FN += 1

y_real: 0.0    y_pred: 1
y_real: 0.0    y_pred: 1
y_real: 1.0    y_pred: 1
y_real: 0.0    y_pred: 1
y_real: 1.0    y_pred: 0
y_real: 0.0    y_pred: 0
y_real: 1.0    y_pred: 1
y_real: 0.0    y_pred: 0
y_real: 1.0    y_pred: 1
y_real: 1.0    y_pred: 1


In [17]:
print("\n ", 'TN =', TN, "\n ",'TP =', TP, "\n ", 'FN =', FN, "\n ", 'FP =', FP)


  TN = 2 
  TP = 4 
  FN = 1 
  FP = 3


In [18]:
conf_matrix = np.array([[TP, FP], [FN, TN]])
conf_matrix

array([[4, 3],
       [1, 2]])

In [19]:
Precision = TP / (TP + FP)
Precision

0.5714285714285714

In [20]:
Recall = TP / (TP + FN)
Recall

0.8

In [21]:
F1 = 2 * Precision * Recall / (Precision + Recall)
F1

0.6666666666666666

6. Могла ли модель переобучиться? Почему?

Могла, т.к. параметры подобраны случайно, а регуляризация не применялась, но при уменьшении количества итераций и увеличении альфы качество результата падает для всех метрик, поэтому в данном случае модель не переобучилась